In [1]:
import time
import numpy as np
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [2]:
opt = webdriver.ChromeOptions()
opt.add_argument("--incognito")
opt.add_argument("disable-infobars")
opt.add_argument("--disable-extensions")
opt.add_argument("--disable-gpu")
opt.add_argument("--disable-dev-shm-usage")
opt.add_argument("--no-sandbox")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument("--headless=new")

driver = webdriver.Chrome(options=opt)
driver.get("https://safebooru.org/index.php?page=tags&s=list&pid=0")
wait = WebDriverWait(driver, 20)

In [3]:
wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "pagination")))

<selenium.webdriver.remote.webelement.WebElement (session="9f420c035396f7f91c91da56c924157b", element="f.F22C7162AB869CD056124B2008BFF586.d.B6FFC1E6B8647F86108BBFB26DD5A1B2.e.7")>

In [4]:
page = driver.page_source
soup = BeautifulSoup(page, "lxml")

In [5]:
tag_table = soup.find(class_="highlightable")

In [9]:
tags = tag_table.find_all("span", class_=["tag-type-copyright", "tag-type-artist", "tag-type-general", "tag-type-character", "tag-type-metadata"])#href=True
#tags = tag_table.find_all("a")

In [34]:
one = tags[0]
one["class"][0][9:]#[21:-1]

'general'

In [8]:
#button_location = driver.find_element(By.CLASS_NAME, "pagination")
#button = button_location.find_element(By.XPATH, '//a[@alt="next"]')
#button.click()

In [9]:
#button_location.text

In [10]:
#soup.find(alt="next") != None

In [11]:
tags = []
scrape = True

while scrape:
    try:
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "pagination")))
        page = driver.page_source
        soup = BeautifulSoup(page, "lxml")
        tag_table = soup.find(class_="highlightable")
        tags += tag_table.find_all("span", class_=["tag-type-copyright", "tag-type-artist", "tag-type-general", "tag-type-character", "tag-type-metadata"])
    
        if soup.find(alt="next") != None:
            button_location = driver.find_element(By.CLASS_NAME, "pagination")
            button = button_location.find_element(By.XPATH, '//a[@alt="next"]')
            button.click()
        else:
            scrape = False
    except TimeoutException:
        driver.refresh()

In [12]:
len(tags)

655914

In [13]:
len(set(tags))

655373

In [14]:
driver.close()

In [15]:
nd_tags = list(set(tags))

In [16]:
url = 'https://safebooru.org'

In [23]:
main_cat = []
names = []
links = []
for i in nd_tags:
    main_cat = i["class"][0][9:]
    get_ = i.find('a')['href'].strip().split('/')[-1]
    link = "{}/{}".format(url, get_)
    names.append(i.text)
    links.append(link)

In [27]:
data = {"Name": names, "Link": links}

In [28]:
df = pd.DataFrame.from_dict(data)
df.head()

,Name,Link
0,mur_cielago_art,https://safebooru.org/index.php?page=post&s=li...
1,godspeedyou,https://safebooru.org/index.php?page=post&s=li...
2,sleeve_lift,https://safebooru.org/index.php?page=post&s=li...
3,donkey_ears,https://safebooru.org/index.php?page=post&s=li...
4,s_minoru,https://safebooru.org/index.php?page=post&s=li...


In [31]:
df.to_csv("Datasets/safebooru_tags_links.csv", index=False)

In [ ]:
# 36 letters for max length to check for one of the four types